# Flight Delay Prediction (Jan 2019)
The dataset contains records gathered by the Bureau of Transportation Statistics (BTS)[9] toprovide historical comparisons of monthly on-time reports filed by large US Airlines.  

_Only datasetsfor 2019 intentionally selected due to the substantial effect of COVID19 in the aviation industry in2020, it is out of scope of this project to analyze this factor._
### Source:Bureau of Transportation Statistics 
### Number of rows:1,984,933
### Dataset features and description:

| Variable name               | Description                                                                                            |
|-----------------------------|--------------------------------------------------------------------------------------------------------|
| Quarter (Time Period)       | Quarter (1-4)                                                                                          |
| Month                       | Month                                                                                                  |
| DayOfWeek                   | Day of Week  (1 - Monday, 2 -Tuesday, 3 - Wendsday)                                                                                          |
| Operating\_Airline          | Carrier Code                                                                                           |
| Origin                      | Origin Airport                                                                                         |
| Dest                        | Destination Airport                                                                                    |
| DepTime                     | Actual Departure Time (local time: hhmm)                                                               |
| DepDelay                    | Difference between scheduled and actual departure time (Minutes)                                       |
| DepDel15                    | Departure Delay Indicator (1=Yes)                                                                      |
| DepartureDelayGroups        | Departure Delay intervals, every (15 minutes from <-15 to >180)                                        |
| TaxiOut                     | Taxi-Out Time (Minutes)                                                                                |
| TaxiIn                      | Taxi-In Time (Minutes)                                                                                 |
| **ArrDelay (target)**          | **Difference in minutes between scheduled and actual arrival time**. *Early arrivals show negative numbers |
| ArrDel15                    | Arrival Delay Indicator, 15 Minutes or More (1=Yes)                                                    |
| ArrivalDelayGroups          | Arrival Delay intervals (15-minutes from <-15 to >180)                                                 |
| Cancelled                   | Cancelled Flight Indicator (1=Yes)                                                                     |
| CancellationCode            | Specifies The Reason For Cancellation                                                                  |
| ActualElapsedTime           | Elapsed Time of Flight, in Minutes                                                                     |
| AirTime                     | Flight Time, in Minutes                                                                                |
| Flights                     | Number of Flights                                                                                      |
| Distance                    | Distance between airports (Miles)                                                                      |
| DistanceGroup               | Distance FLight Segment, (every 250 Miles)                                                             |
| CarrierDelay                | Delay by Carrier (Minutes)                                                                             |
| NASDelay                    | Delay by NAS (Minutes)                                                                                 |
| SecurityDelay               | Delay by Security (Minutes)                                                                            |
| LateAircraftDelay           | Delay by Late Aircraft (in Minutes)                                                                    |
| WeatherDelay                | Delay caused by Weather (Minutes)                                                                      |

In [ ]:
#Import basic libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#Setup pandas display parameters
pd.options.display.max_columns = 50
pd.options.display.max_rows = 200
pd.options.display.max_colwidth= 50
pd.options.display.precision = 3

In [ ]:
#Define initial model parameters
cv_n_split = 3
random_state = 42
test_train_split = 0.25
sample = True
sample_size = 0.1

In [ ]:
#Setup  Pplotting enviroment
sns.set()
sns.set_context('notebook',rc={"grid.linewidth": 5})
sns.set_style("whitegrid")
colors = ["#345E6F","#264653","#287271","#2a9d8f","#e9c46a","#efb366","#f4a261","#ee8959","#e76f51","#e87153","#e97c61", '#902C14']
colors = ["#5998B1","#406F82","#264653","#287271","#2a9d8f","#8AB17D","#E9C46A","#EFB366","#F4A261","#E76F51","#C53D1B","#923B26"]
bin_colors = ["#264653","#2A9D8F","#ee8959","#e97c61"]
sns.set_palette(sns.color_palette(colors))
mul_palette = sns.color_palette(colors)
bin_palette = sns.color_palette(bin_colors)
sns.set(rc={'figure.figsize':(10,5)}, font_scale=1.5)
sns.set_style({'axes.facecolor':'white', 'grid.color': '.8','grid.linestyle': '--'})

# Data Selection and Cleaning

In [ ]:
#data = pd.read_csv("../1019415451_T_ONTIME_MARKETING-4.csv")
df01_csv = pd.read_csv("../../Downloads/ontime-Jan2019.csv")
if sample:
    df01_csv = df01_csv.sample(frac = sample_size, replace=True, random_state=random_state)
print ("Imported {} with {} variables".format(df01_csv.shape[0],df01_csv.shape[1]))

In [ ]:
df01_csv.columns = map(str.lower, df01_csv.columns)  #Set column names to lowercase
df01_csv.head()

In [ ]:
df01_csv.dtypes.sort_values()

In [ ]:
#Review dataframe columns summary for diagnostics and cleaning
total_rows_csv = df01_csv.shape[0]
pd.DataFrame({ 
            'unicos':df01_csv.nunique(),
            'count': df01_csv.count(),
            'missing total': df01_csv.isna().sum(),
            'missing %': (df01_csv.isna().sum()/total_rows_csv)*100,
            'type':df01_csv.dtypes})

In [ ]:
#Drop non-usable columns
df01_csv.drop(['unnamed: 30','cancellation_code','tail_num'],axis='columns', inplace=True)
print ("Imported {} with {} variables".format(df01_csv.shape[0],df01_csv.shape[1]))

In [ ]:
#Check for null values on cancelled
pd.crosstab(df01_csv.dep_delay.isna(), df01_csv.cancelled).sort_values(by=1,ascending=False)

In [ ]:
#Check for null values on cancelled
pd.crosstab(df01_csv.dep_delay.isna(), df01_csv.diverted).sort_values(by=1,ascending=False)

In [ ]:
#Most of Null values belong to either cancelled or diverter flights, thus, we will drop those rows from the dataset
df_delayed = df01_csv[(df01_csv['cancelled'] == 0) & (df01_csv['diverted']==0)]
df_delayed.shape

In [ ]:
#Replace with 0 all NA values on delay 
df_delayed['carrier_delay'] = df_delayed['carrier_delay'].fillna(0)
df_delayed['weather_delay'] = df_delayed['weather_delay'].fillna(0)
df_delayed['nas_delay'] = df_delayed['nas_delay'].fillna(0)
df_delayed['security_delay'] = df_delayed['security_delay'].fillna(0)
df_delayed['late_aircraft_delay'] = df_delayed['late_aircraft_delay'].fillna(0)

In [ ]:
df_delayed.head()

In [ ]:
#Fix data types for categorical columns
for col in ['day_of_month','day_of_week','op_unique_carrier','origin','dest','dep_delay_group','dep_time_blk','arr_time_blk' ,'arr_delay_group']:
    df_delayed[col] = df_delayed[col].astype('category')

In [ ]:
#Review dataframe columns summary for diagnostics and cleaning
total_rows_clean = df_delayed.shape[0]
pd.DataFrame({ 
            'unicos':df_delayed.nunique(),
            'count': df_delayed.count(),
            'missing total': df_delayed.isna().sum(),
            'missing %': (df_delayed.isna().sum()/total_rows_clean)*100,
            'type':df_delayed.dtypes})

In [ ]:
#Total droped rows
rows_dropped = total_rows_csv - total_rows_clean
rows_dropped_proportion = rows_dropped/total_rows_csv
print("Rows Dropped: {} ({:.2%})".format(rows_dropped,rows_dropped_proportion))

# Data Construction
Derived Attibutes and Records

In [ ]:
#Import dataset for bussi airports, contains airport code and Total Enplaned Passengers in 2019
top_airports_csv = pd.read_csv("datasets/topAirports2.csv")
top_airports_csv['total_enplaned'].describe()

In [ ]:
top_airports_csv.head()

In [ ]:
total_emplaned = top_airports_csv.total_enplaned.sum()
top_airports_csv['enplaned_percentage'] = top_airports_csv['total_enplaned']/total_emplaned
top_airports_csv.head()

In [ ]:
top10 = top_airports_csv.nlargest(10, 'total_enplaned')
top10['airport_code']

In [ ]:
df_delayed['origin_top10'] = df_delayed.origin.isin(top10['airport_code'].tolist())
df_delayed['dest_top10'] = df_delayed.dest.isin(top10['airport_code'].tolist())

In [ ]:
top_airports_csv.isna().sum()

In [ ]:
top_airports = top_airports_csv.set_index('airport_code')
top_airports.drop(['total_enplaned'],axis='columns', inplace=True) # Drop total_enplaned  column before megre
top_airports.head()

Create new column that contains the percentage of the emplaned passangers in 2019 for origin and destination

In [ ]:
#merge the airport dataset on destination to create new column with percentage of emplaned per airport
df_delayed = df_delayed.merge(top_airports, how='left', left_on='dest', right_index=True)
 #Rename columns to match the main data frame
df_delayed.rename(columns = {'enplaned_percentage':'dest_emplaned'}, inplace = True)
#fill with 0 the cells corresponding to airports not in the top airports list 
df_delayed.dest_emplaned.fillna(0,inplace=True)

In [ ]:
#merge the airport dataset on origin to create new column with percentage of emplaned per airport
df_delayed = df_delayed.merge(top_airports, how='left', left_on='origin', right_index=True)
#Rename columns to match the main data frame
df_delayed.rename(columns = {'enplaned_percentage':'origin_emplaned'}, inplace = True) 
#fill with 0 the cells corresponding to airports not in the top airports list 
df_delayed.origin_emplaned.fillna(0,inplace=True) 
#Print the new data frame
df_delayed.head()

Apply frequency encoding to categorical variables

In [ ]:
#Frecuqncy encoding Origin
origin_freq_encoding = (df_delayed.groupby('origin').size()) / len(df_delayed)
df_delayed['origin_freq_encoding'] = df_delayed['origin'].apply(lambda x : origin_freq_encoding[x])

In [ ]:
#Frecuqncy encoding Destination
dest_freq_encoding = (df_delayed.groupby('dest').size()) / len(df_delayed)
df_delayed['dest_freq_encoding'] = df_delayed['dest'].apply(lambda x : dest_freq_encoding[x])

In [ ]:
#Frecuqncy encoding Departure time block
dept_blk_freq = (df_delayed.groupby('dep_time_blk').size()) / len(df_delayed)
df_delayed['dept_blk_freq'] = df_delayed['dep_time_blk'].apply(lambda x : dept_blk_freq[x])


In [ ]:
#Frecuqncy encoding Departure time block
dept_blk_freq = (df_delayed.groupby('arr_time_blk').size()) / len(df_delayed)
df_delayed['arr_blk_freq'] = df_delayed['arr_time_blk'].apply(lambda x : dept_blk_freq[x])

In [ ]:
#Frecuqncy encoding Departure time block
dept_blk_freq = (df_delayed.groupby('op_unique_carrier').size()) / len(df_delayed)
df_delayed['op_carrier_freq'] = df_delayed['op_unique_carrier'].apply(lambda x : dept_blk_freq[x])

In [ ]:
#Frecuqncy encoding day_of_week
freq_encoding = (df_delayed.groupby('day_of_week').size()) / len(df_delayed)
df_delayed['day_of_week_freq'] = df_delayed['day_of_week'].apply(lambda x : freq_encoding[x-1])

In [ ]:
#Frecuqncy encoding day_of_week
freq_encoding = (df_delayed.groupby('day_of_month').size()) / len(df_delayed)
df_delayed['day_of_month_freq'] = df_delayed['day_of_month'].apply(lambda x : freq_encoding[x-1])

In [ ]:
df_delayed = df_delayed.drop(['origin','dest','arr_time_blk','op_unique_carrier','cancelled','diverted','arr_delay_group'], axis='columns', inplace=False)

In [ ]:
df_delayed.head()

# Data Exploration and Visualization

In [ ]:
ax = sns.histplot(data = top_airports_csv , x ='total_enplaned', kde=True,bins=10)
ax.axes.yaxis.set_visible(False)
ax.set(xlabel="",title='Airport Enplaned Histogram')

In [ ]:
ax = sns.barplot(data = top_airports_csv , y = 'total_enplaned', x ='airport_code',palette = mul_palette)
ax.axes.yaxis.set_visible(False)
ax.set(xlabel="",title='Airport Enplaned Histogram')
_ = ax.set_xticklabels(ax.get_xticklabels(),rotation=90)

In [ ]:
fig, axes = plt.subplots(1, 2)
sns.set_palette("bright")
sns.histplot(data=df_delayed, x="arr_delay",kde=True, bins = 100,ax = axes[0])
sns.boxplot(data=df_delayed, y="arr_delay",ax=axes[1])
axes[0].set(xlabel = "", ylabel = "",title = 'Arrival delay Histogram',)
axes[1].set(xlabel = "", ylabel = "",title = 'Arrival delay Boxplot',)
axes[0].axes.yaxis.set_visible(False)

In [ ]:
ax = sns.scatterplot(x="dep_delay", y="arr_delay", hue = 'arr_del15', data = df_delayed, palette=('YlGn'))
ax.set(xlabel = "Departure Delay", ylabel = "Arrival Delay",title = 'Departure delay vs arrival delay (Minutes)')
ax.legend([ 'Delayed','On-Time'])
ax.axes.yaxis.set_ticks([])

In [ ]:
df_delayed.dep_delay.describe()

In [ ]:
#Filter any rtows above the 95 percentile
df_delayed_outliers = df_delayed[df_delayed.dep_delay<df_delayed.dep_delay.quantile(.90)]
#Total droped rows
total_rows = df_delayed.shape[0]
total_rows_clean = df_delayed_outliers.shape[0]
rows_dropped = total_rows - total_rows_clean
rows_dropped_proportion = rows_dropped/total_rows
print("Rows Dropped: {} ({:.2%})".format(rows_dropped,rows_dropped_proportion))

In [ ]:
ax = sns.scatterplot(x="dep_delay", y="arr_delay", hue = 'arr_del15', data = df_delayed_outliers, palette=('YlGn'))
ax.set(xlabel = "Departure Delay", ylabel = "Arrival Delay",title = 'Departure delay vs arrival delay (Minutes)')
ax.legend([ 'Delayed','On-Time'])
ax.axes.yaxis.set_ticks([])

In [ ]:
fig, axes = plt.subplots(1, 2)
sns.set_palette("bright")
sns.histplot(data=df_delayed_outliers, x="arr_delay",kde=True, bins = 100,ax = axes[0])
sns.boxplot(data=df_delayed_outliers, y="arr_delay",ax=axes[1])
axes[0].set(xlabel = "", ylabel = "",title = 'Arrival delay Histogram',)
axes[1].set(xlabel = "", ylabel = "",title = 'Arrival delay Boxplot',)
axes[0].axes.yaxis.set_visible(False)

In [ ]:
df_delayed_outliers.dep_delay.describe()

## Calculate correlation coefficient

In [ ]:
np.corrcoef(df_delayed['dep_delay'], df_delayed['arr_delay'])

In [ ]:
np.corrcoef(df_delayed_outliers['dep_delay'], df_delayed_outliers['arr_delay'])

In [ ]:
df_delayed.dtypes

In [ ]:
num_columns = list(df_delayed.select_dtypes(include=np.number).columns)
num_columns.remove("arr_delay")
num_columns.remove("arr_delay_new")
num_columns.remove("arr_del15")
len(num_columns)

In [ ]:
cols = 2
rows = 8
fig, axs = plt.subplots(ncols=cols,nrows=rows,figsize=(10,16))
for i in range (cols):
    for j in range(rows):
        index = i*rows+j
        if index < len(num_columns):
            sns.regplot(x=num_columns[i*rows+j], y = "arr_delay", data = df_delayed,ax=axs[j,i],scatter_kws={"color": "#2A9D8F"}, line_kws={"color": "#264653"})
            axs[j,i].set(xlabel = "",title = num_columns[i*rows+j])
            axs[j,i].axes.yaxis.set_visible(False)
            axs[j,i].axes.xaxis.set_visible(False)

In [ ]:
ax = sns.scatterplot(x="dep_time_blk", y="arr_delay", data = df_delayed,palette=('YlGn'))
ax.set(xlabel = "Departure Time Block", ylabel = "",title = 'Departure Time vs Arrival delay (Minutes)')
#set labels friendly name
_ = ax.set_xticklabels(list(df_delayed.dep_time_blk.unique()),rotation=90)
#Hide Y labels
#ax.axes.yaxis.set_visible(False)

In [ ]:
ax = sns.countplot(x="dep_time_blk", hue = 'arr_del15', data = df_delayed,palette = bin_palette)
ax.set(xlabel = "", ylabel = "",title = 'Arrival delay vs Departure time block')
#set labels friendly name
ax.set_xticklabels(list(df_delayed.dep_time_blk.unique()),rotation=90)
ax.legend(['On-Time', 'Delayed'])

In [ ]:
#create plot
ax = sns.countplot(y = 'day_of_week',  data = df_delayed[df_delayed['arr_del15']>0], palette = mul_palette)
#Set Title
ax.set(xlabel = "",ylabel = "",title = 'Delayed Flights per day of the week')
#set labels friendly name
ax.set_yticklabels(['Monday','Tuesday','Wensday','Thursday','Friday','Saturday','Sunday'])
#ax.set_xticklabels(['On Time','Delayed'])
#Hide Y labels
ax.axes.yaxis.set_visible(True)

In [ ]:
#create plot
ax = sns.countplot(x = 'day_of_month' , data = df_delayed[df_delayed['arr_del15']>0],palette = mul_palette)
#Set Title
ax.set(xlabel = "",title = 'Delayed flights per day of the month')
#set labels friendly name
#ax.set_xticklabels(['On Time','Delayed'])
#Hide Y labels
ax.axes.yaxis.set_visible(False)

# Selecting variables for model

In [ ]:
plt.figure(figsize=(16, 6))
# define the mask to set the values in the upper triangle to True
mask = np.triu(np.ones_like(df_delayed.corr(), dtype=np.bool))
heatmap = sns.heatmap(df_delayed.corr(), mask=mask, vmin=-1, vmax=1, cmap='BrBG')
heatmap.set_title('Triangle Correlation Heatmap', fontdict={'fontsize':18}, pad=16);

In [ ]:

fig, ax = plt.subplots(1, 2)
fig.set_size_inches(8,10)
sns.heatmap(df_delayed.corr()[['arr_delay']].sort_values(by='arr_delay', ascending=False), vmin=-1, vmax=1, annot=True, cmap='BrBG',ax=ax[0],cbar=False)
sns.heatmap(df_delayed_outliers.corr()[['arr_delay']].sort_values(by='arr_delay', ascending=False), vmin=-1, vmax=1, annot=True, cmap='BrBG',ax=ax[1],cbar=True)
ax[0].set_title('Features Correlation', fontdict={'fontsize':16}, pad=16);
ax[1].set_title('Features Correlation (removed Outliers)', fontdict={'fontsize':16}, pad=16);
ax[1].axes.yaxis.set_visible(False)

In [ ]:
df_train = df_delayed[['dep_delay','carrier_delay','late_aircraft_delay','dep_del15','nas_delay','weather_delay','arr_delay','arr_delay_new']]
df_train_outliers = df_delayed_outliers[['dep_delay','carrier_delay','late_aircraft_delay','dep_del15','nas_delay','weather_delay','arr_delay','arr_delay_new']]

In [ ]:

fig, ax = plt.subplots(1, 2)
fig.set_size_inches(8,10)
sns.heatmap(df_train.corr()[['arr_delay']].sort_values(by='arr_delay', ascending=False), vmin=-1, vmax=1, annot=True, cmap='BrBG',ax=ax[0],cbar=False)
sns.heatmap(df_train_outliers.corr()[['arr_delay']].sort_values(by='arr_delay', ascending=False), vmin=-1, vmax=1, annot=True, cmap='BrBG',ax=ax[1],cbar=True)
ax[0].set_title('Features Correlation', fontdict={'fontsize':16}, pad=16);
ax[1].set_title('Features Correlation (removed Outliers)', fontdict={'fontsize':16}, pad=16);
ax[1].axes.yaxis.set_visible(False)

In [ ]:
fig, ax = plt.subplots(1, 2)
sns.regplot(x="dep_delay", y="arr_delay", data=df_train, scatter_kws={"color": "#2A9D8F"}, line_kws={"color": "#264653"},ax=ax[0])
sns.regplot(x="dep_delay", y="arr_delay", data=df_train_outliers, scatter_kws={"color": "#2A9D8F"}, line_kws={"color": "#264653"},ax=ax[1])
ax[0].set(xlabel = "", ylabel = "")
ax[1].set(xlabel = "", ylabel = "")

In [ ]:
df_train.agg(['skew', 'kurtosis']).transpose()

In [ ]:
df_train_outliers.agg(['skew', 'kurtosis']).transpose()

In [ ]:
# Python Square root transformation
df_train['arr_sqrt'] = df_train['arr_delay'].apply(np.sqrt)
df_train['arr_log'] = np.log(0.001+df_train['arr_delay_new'])

df_train_outliers['arr_sqrt'] = df_train_outliers['arr_delay'].apply(np.sqrt)


In [ ]:
df_train.head()

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(12, 8))
sns.histplot(data = df_train , x ='arr_delay', kde=True,bins=10,ax=ax[0,0])
sns.histplot(data = df_train , x = 'arr_sqrt', kde=True,bins=10,ax=ax[0,1])
sns.histplot(data = df_train , x = 'arr_log', kde=True,bins=10,ax=ax[1,0])
sns.histplot(data = df_train_outliers , x = 'arr_sqrt', kde=True,bins=10,ax=ax[1,1])
ax[0,0].axes.yaxis.set_visible(False)
ax[0,0].set(xlabel="",title='Arrival Delay')
ax[0,1].axes.yaxis.set_visible(False)
ax[0,1].set(xlabel="",title='Arrival Delay Squared')
ax[1,0].axes.yaxis.set_visible(False)
ax[1,1].axes.yaxis.set_visible(False)
ax[1,0].set(xlabel="",title='Arrival Delay Log + 1')
ax[1,1].set(xlabel="",title='Arrival Delay Sqrt (outliers)')

# Model Building

In [ ]:
import numpy as np
from scipy import stats
from statistics import mean
#Import ML models t be used
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_regression
from xgboost import XGBRegressor
import xgboost as xgb
#Libraries for model selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import learning_curve
from sklearn.model_selection import StratifiedKFold
#Libraries for model evaluation
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_validate



In [ ]:
# Funtion to run the m odel and calculate the various evaluation metrics and plor the corresponding plots
def run_model(model, X_train, y_train, X_test, y_test, verbose=True, desc = 'No name model'):
        model.fit(X_train,y_train)  # perform linear regression
        y_pred = model.predict(X_test)
        r2 =  r2_score(y_test, y_pred)
        #adj_r2 = 1 - (1-r2)*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)
        #adj_r2 = (1 - (1 - r2) * ((X_train.shape[0] - 1) / (X_train.shape[0] - X_train.shape[1] - 1)))
        adj_r2 = 1 - (1-r2)*(len(X_train) - 1) / (len(X_train) - (X_train.shape[1] - 1) - 1)
        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        cross_val_score_mean = mean(cross_val_score(model,X_train,y_train,cv=cv_n_split))
        #print('R2 score: {}'.format(r2))
        print('R2 score: %.2f' % r2)
        print('Adj. R2 score: %.2f' % adj_r2)
        print('Mean absolute error: %.2f' % mae  )
        print('Mean squared error: %.2f' % mse)
        print('Root Mean Squared Error: {}'.format(rmse))
        print("Avg CrossValid Score  : {:.2%} ({} Folds)".format(cross_val_score_mean,cv_n_split))
        if type(model) == xgb.sklearn.XGBRegressor:
            print("________")
        else:
        # The coefficients
            data = {"variables": list(X_train.columns),
                    "Coefficients": model.coef_}
            df_coefficients = pd.DataFrame(data)
            print('Intercept: ', model.intercept_)          
            print('Coefficients:')
            print(df_coefficients)
        residuals = y_test - y_pred
        model_norm_residuals_abs_sqrt=np.sqrt(np.abs(residuals))

        model_results[len(model_results)] = {
                                            'model'                 :type(model),
                                            'description'           :desc,
                                            'R2-score'              :r2,
                                            'AdjR2-score'           :adj_r2,
                                            'MAE'                   :mae,
                                            #'MSE'                   :mse,
                                            'RMSE'                  :rmse,
                                            'crossval_mean'  :cross_val_score_mean
                                            }



        #x = cross_validate(model,X_train,y_train,cv=cv_n_split)
        #print(x['test'])
        #mean(cross_validate(model,X_train,y_train,cv=cv_n_split))
        #print("Avg CrossValid Score  : {:.2%} ({} Folds)".format(cross_val_score_mean,cv_n_split))
        
        fig, ax = plt.subplots(2, 2, figsize=(15, 12))
        #ax[0] = plt.subplot(221)
        stats.probplot(residuals, dist="norm", plot=ax[0,1])
        sns.regplot(y_pred, model_norm_residuals_abs_sqrt,
                    scatter=True,
                    lowess=True,
                    color='#2A9D8F',
                    line_kws={'color': '#264653', 'lw': 1, 'alpha': 0.8},ax=ax[0,0])
        sns.residplot(y_test,y_pred, color='#2A9D8F',lowess=True,
                                        line_kws={'color': '#264653', 'lw': 1, 'alpha': 1},ax=ax[1,0])

        sns.distplot(y_test, kde=True, norm_hist=True, hist=False,color='#2A9D8F', label='Standard Normal', ax = ax[1,1])                                    
        sns.distplot(y_pred, kde=True, norm_hist=True, hist=False,color='#264653', label='Skew Normal $\\alpha = 5$',ax=ax[1,1])
        ax[1,1].legend(['y_test', 'y_pred'])
        #sns.regplot(x=y_test, y=y_pred,ax=ax[1,1],scatter_kws={"color": "#2A9D8F"}, line_kws={"color": "#264653"})
        ax[0,0].set(xlabel="Fitted value",ylabel='Standarized residuals',title='Scale-Location plot')
        ax[0,1].set(xlabel="Theoretichal Values",ylabel='Ordered Values',title='Normal Q-Q Plot')
        ax[1,0].set(xlabel="Obsevation #",ylabel='Error',title='Model Residuals')
        ax[1,1].set(xlabel="Arrival Delay",ylabel='',title='Comparison of distributions')
        #ax[1,1].set(xlabel="Arrival Delay",ylabel='',title='Prediction')

def printModelResults(results):
    df_results = pd.DataFrame.from_dict(results)
    df_results = df_results.T
    print(df_results)
    heat = df_results[['description','R2-score','AdjR2-score','MAE','RMSE','crossval_mean']]
    heat = heat.set_index('description')
    heat = heat.sort_values(by='R2-score', ascending=False)
    for col in heat.columns:
        heat[col] = heat[col].astype(float)
    fig, ax = plt.subplots(1)
    fig.set_size_inches(15, 8)
    fig.suptitle('Model Comparisson')
    sns.heatmap(heat,annot=True, ax = ax )

In [ ]:
list(df_train.columns)

In [ ]:
y = df_train.pop('arr_delay')
y_sqrt = df_train.pop('arr_sqrt')
y_log = df_train.pop('arr_log')
y_new = df_train.pop('arr_delay_new')
X = df_train
print(X.shape)
print(y.shape)

In [ ]:
model_results = {} # Create empty list with models results

In [ ]:
#split into raining and test
X_train,X_test, y_train,y_test = train_test_split(X , y , random_state = random_state, shuffle = True, test_size = test_train_split)
#Create Linear Regression model
model_lr1 = LinearRegression()
run_model(model_lr1, X_train, y_train, X_test, y_test,desc='LR - y')

In [ ]:
#split into raining and test on delay_new
X_train,X_test, y_train,y_test = train_test_split(X , y_new , random_state = random_state, shuffle = True, test_size = test_train_split)
#Create Linear Regression model
model_lr1 = LinearRegression()
run_model(model_lr1, X_train, y_train, X_test, y_test,desc='LR - y_new')

In [ ]:
#split into raining and test on y log
X_train,X_test, y_train,y_test = train_test_split(X , y_log , random_state = random_state, shuffle = True, test_size = test_train_split)
#Run Linear Regression model
run_model(LinearRegression(), X_train, y_train, X_test, y_test,desc='LR - Log(y)')

In [ ]:
# Fill with 0 NA values on y_sqrt
y_sqrt = y_sqrt.fillna(0)

In [ ]:
#split into raining and test on y square
X_train,X_test, y_train,y_test = train_test_split(X , y_sqrt , random_state = random_state, shuffle = True, test_size = test_train_split)
#Create Linear Regression model
run_model(LinearRegression(), X_train, y_train, X_test, y_test,desc='LR - Sqrt(y)')

In [ ]:
#split into raining and test
X_train,X_test, y_train,y_test = train_test_split(X , y , random_state = random_state, shuffle = True, test_size = test_train_split)
#Run xgboost Regression model
run_model(xgb.XGBRegressor(objective="reg:squarederror"), X_train, y_train, X_test, y_test, desc='XGBoost - y')

In [ ]:
#split into raining and test
X_train,X_test, y_train,y_test = train_test_split(X , y_new , random_state = random_state, shuffle = True, test_size = test_train_split)
#Run xgboost Regression model
run_model(xgb.XGBRegressor(objective="reg:squarederror"), X_train, y_train, X_test, y_test, desc='XGBoost - y_new')

In [ ]:
printModelResults(model_results)

## Models on  DF removed outliers

In [ ]:
y = df_train_outliers.pop('arr_delay')
y_new = df_train_outliers.pop('arr_delay_new')
y_sqrt = df_train_outliers.pop('arr_sqrt')
X = df_train_outliers
print(X.shape)
print(y.shape)

In [ ]:
#split into raining and test on delay_new
X_train,X_test, y_train,y_test = train_test_split(X , y_new , random_state = random_state, shuffle = True, test_size = test_train_split)
#Create Linear Regression model
model_lr1 = LinearRegression()
run_model(model_lr1, X_train, y_train, X_test, y_test,desc='LR - y_new (outliers removed)')

In [ ]:
#split into raining and test
X_train,X_test, y_train,y_test = train_test_split(X , y_new , random_state = random_state, shuffle = True, test_size = test_train_split)
#Run xgboost Regression model
run_model(xgb.XGBRegressor(objective="reg:squarederror"), X_train, y_train, X_test, y_test, desc='XGBoost - y_new (outliers removed)')

In [ ]:
# Fill with 0 NA values on y_sqrt
y_sqrt = y_sqrt.fillna(0)
#split into raining and test
X_train,X_test, y_train,y_test = train_test_split(X , y_sqrt , random_state = random_state, shuffle = True, test_size = test_train_split)
#Run xgboost Regression model
run_model(xgb.XGBRegressor(objective="reg:squarederror"), X_train, y_train, X_test, y_test, desc='XGBoost - y_sqrt (outliers removed)')

In [ ]:
printModelResults(model_results)

In [ ]:
linear_regressor = LinearRegression()
linear_regressor.fit(X_train, y_train )  # perform linear regression
y_pred = linear_regressor.predict(X_test)  # make predictions

# The coefficients
print('Coefficients: ', linear_regressor.coef_)
print('Intercept: ', linear_regressor.intercept_)
print('R-Squared :', linear_regressor.score(X_test, y_test))

# The mean squared error
print('Mean squared error: %.2f' % mean_squared_error(y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f' % r2_score(y_test, y_pred))


In [ ]:
sns.distplot(y_pred, hist = False, color = 'r', label = 'Predicted Values')
sns.distplot(y_test, hist = False, color = 'b', label = 'Actual Values')
plt.title('Actual vs Predicted Values', fontsize = 16)
plt.xlabel('Values', fontsize = 12)
plt.ylabel('Frequency', fontsize = 12)
plt.legend(loc = 'upper left', fontsize = 13)

In [ ]:

stats.probplot(residuals, dist="norm", plot=plt)
plt.title("Normal Q-Q Plot")

In [ ]:
model_norm_residuals_abs_sqrt=np.sqrt(np.abs(residuals))
sns.regplot(y_pred, model_norm_residuals_abs_sqrt,
              scatter=True,
              lowess=True,
              line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8})
plt.ylabel("Standarized residuals")
plt.xlabel("Fitted value")

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(15, 12))
#ax[0] = plt.subplot(221)
stats.probplot(residuals, dist="norm", plot=ax[0,1])
sns.regplot(y_pred, model_norm_residuals_abs_sqrt,
              scatter=True,
              lowess=True,
              color='#2A9D8F',
              line_kws={'color': '#264653', 'lw': 1, 'alpha': 0.8},ax=ax[0,0])
sns.residplot(y_test,y_pred, color='#2A9D8F',lowess=True,
                                  line_kws={'color': '#264653', 'lw': 1, 'alpha': 1},ax=ax[1,0])

sns.distplot(y_test, kde=False, norm_hist=True, color='#2A9D8F', label='Standard Normal', ax = ax[1,1])                                    
sns.distplot(y_pred, kde=False, norm_hist=True, color='#264653', label='Skew Normal $\\alpha = 5$',ax=ax[1,1])
ax[0,0].set(xlabel="Fitted value",ylabel='Standarized residuals',title='Scale-Location plot')
ax[0,1].set(xlabel="Theoretichal Values",ylabel='Ordered Values',title='Normal Q-Q Plot')
ax[1,0].set(xlabel="Obsevation #",ylabel='Error',title='Model Residuals')
ax[1,1].set(xlabel="Arrival Delay",ylabel='',title='Comparison of distributions')

In [ ]:
sns.regplot(x=y_test, y=y_pred);